In [17]:
import pandas as pd
import numpy as np
from joblib import load
import glob
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import random
import sklearn.model_selection as ms
from sklearn import ensemble

In [18]:
path = '../data/weatherdata'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

weather_df = pd.concat(li, axis=0, ignore_index=True)

/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (43,44,48,53,57,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (45,46,50,55,59,88,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (42,43,44,45,56,57,64,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (42,43,44,45,49,54,58,59) have mi

In [19]:
def downsampling():
    sampled_list = []
    n = 0
    while n <= 3:
        print(n)
        p = 0.01  # 1% of the lines
        # keep the header, then take only 1% of lines
        # if random from [0,1] interval is greater than 0.01 the row will be skipped
        df = pd.read_csv('../data/stations_cleaned.csv.gz', skiprows=lambda i: i>0 and random.random() > p,
                         parse_dates = ['date'])
        df['Sample_num'] = n
        sampled_list.append(df)
        n += 1
    return pd.concat(sampled_list)

In [20]:
dock_sample = downsampling()

0


/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


1
2
3


In [5]:
#dock = pd.read_csv('../data/stations_cleaned.csv.gz')

In [21]:
def to_milesperhour(num):
    """A function to convert wind speed from meters/second to miles/hour. The conversion is done by using the 
    above formula
    """
    convert = (num * 3600)/(1000 * 1.6)
    return convert

In [22]:
def clean_weatherdata(df):
    """This function will clean the weather data from any given year or years (the merged weather data)
    Weather data was obtained from NOAA( National Oceanic and Atmospheric Administration ) 
    https://www.ncei.noaa.gov/data/global-hourly/archive/csv/.
    Additional websites were also used as references in order to interpret the numbers from the data and to
    engineer new features such as windy and rainy. The column windy and its values (breeze, gale, force...)
    were all based on information from NOAA https://www.weather.gov/pqr/wind. 
    https://www.visualcrossing.com/resources/documentation/weather-data/how-we-process-integrated-surface-database-historical-weather-data/
    was used to interpret the numbers of the data.
    """
    # Make a copy
    nyweather= df.copy()
    
    # Select the columns that will be used
    nyweather = nyweather[['DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'WND', 'TMP',
             'DEW', 'SLP', 'AA1','AA2']]
    
    # Clean DATE column
    nyweather['DATE'] = nyweather['DATE'].str.replace('T', ' ')
    
    # Clean TMP (temperature) column. Please refer to 
    # Interpreting the numbers are based on 
    #https://www.visualcrossing.com/resources/documentation/weather-data/how-we-process-integrated-surface-database-historical-weather-data/

    nyweather['TMP'] = nyweather['TMP'].astype(str)
    nyweather['TMP'] = nyweather['TMP'].str.slice(0, -2)
    nyweather['TMP'] = nyweather['TMP'].str.replace('+','')
    nyweather = nyweather.drop(nyweather[nyweather.TMP == '9999'].index)
    nyweather['TMP'] = nyweather['TMP'].astype(int)
    nyweather['TMP'] = nyweather['TMP']/10
    print('complete part 1')
    
    # Clean WND column
    # Please refer to https://www.weather.gov/pqr/wind
    nyweather['WND'] = nyweather['WND'].astype(str)
    nyweather = pd.concat([nyweather, nyweather.WND.str.split(',', expand = True)],1)
    nyweather = nyweather.rename(columns={3:'wind_speed'})
    nyweather['wind_speed'] = nyweather['wind_speed'].astype(int)
    nyweather['wind_speed'] = nyweather['wind_speed']/10
    nyweather['wind_speed'] = nyweather['wind_speed'].map(to_milesperhour)
    
    # Engineered a new feature called windy
    nyweather['windy'] = nyweather['wind_speed'].apply(lambda x: 'calm' if x < 4 else 'breeze' if x < 12 else \
                                        'moderate breeze' if x < 24 else 'strong breeze' if x < 31 else \
                                        'gale' if x < 63 else 'storm force')
    nyweather = nyweather[['DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'WND', 'TMP',\
                           'DEW', 'SLP', 'AA1','AA2', 'wind_speed', 'windy']]
    
    print('complete part 2')
    
    # Clean the AA1 column that has precipitation information
    nyweather['AA1'] = nyweather['AA1'].fillna('0,0,0,0')
    nyweather['AA1'] = nyweather['AA1'].astype(str)
    nyweather = pd.concat([nyweather, nyweather.AA1.str.split(',', expand = True)],1)
    nyweather = nyweather.rename(columns={1:'precipitation'})
    nyweather['precipitation'] = nyweather['precipitation'].astype(int)
    nyweather['precipitation'] = nyweather['precipitation']/10
    nyweather = nyweather[['DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'WND', 'TMP',\
                           'DEW', 'SLP', 'wind_speed', 'windy', 'precipitation']]
    
    # Engineered a new feature called rainy
    nyweather['rainy'] = nyweather['precipitation'].apply(lambda x: 'rainy' if x > 0 else 'not rainy')
    
    
    # Clean DATE column
    # Generate new features - month, date, hour and month2
    nyweather['DATE'] = nyweather['DATE'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    nyweather['month'] = nyweather.DATE.dt.month
    nyweather['date'] = nyweather.DATE.dt.date
    nyweather['hour'] = nyweather.DATE.dt.hour
    nyweather['month2'] = nyweather.DATE.dt.month.apply(lambda x: 'Jan' if x == 1 else 'Feb' if x == 2 else \
    'Mar' if x == 3 else 'Apr' if x == 4 else 'May' if x == 5 else 'Jun' if x == 6 else 'Jul' if x == 7 else \
    'Aug' if x == 8 else 'Sep' if x == 9 else 'Oct' if x == 10 else 'Nov' if x == 11 else 'Dec' )
    
    # Generate cleaned dataframe
    return nyweather

In [23]:
weather_df = clean_weatherdata(weather_df)

complete part 1
complete part 2


In [24]:
dock_sample

,dock_id,dock_name,date,hour,minute,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_status,time,dayofweek,dayofweek2,season,Sample_num
0,72,W 52 St & 11 Ave,2015-03-01,6,24,8,30,38,40.767272,-73.993929,1,1,Empty Alert,6:24,6,Sunday,spring,0
1,72,W 52 St & 11 Ave,2015-03-01,10,5,9,29,38,40.767272,-73.993929,1,1,Empty Alert,10:5,6,Sunday,spring,0
2,72,W 52 St & 11 Ave,2015-03-02,17,34,9,29,38,40.767272,-73.993929,1,1,Empty Alert,17:34,0,Monday,spring,0
3,72,W 52 St & 11 Ave,2015-03-02,18,7,9,30,39,40.767272,-73.993929,1,1,Empty Alert,18:7,0,Monday,spring,0
4,72,W 52 St & 11 Ave,2015-03-03,17,20,8,30,38,40.767272,-73.993929,1,1,Empty Alert,17:20,1,Tuesday,spring,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323283,3533,St. Nicholas Ave & W 126 St,2019-01-24,18,39,4,29,34,40.811432,-73.951878,1,1,Empty Alert,18:39,3,Thursday,winter,3
323284,3533,St. Nicholas Ave & W 126 St,2019-01-27,5,5,4,29,34,40.811432,-73.951878,1,1,Empty Alert,5:5,6,Sunday,winter,3
323285,3533,St. Nicholas Ave & W 126 St,2019-01-30,12,41,7,27,34,40.811432,-73.951878,1,1,Empty Alert,12:41,2,Wednesday,winter,3
323286,3534,Frederick Douglass Blvd & W 117 St,2019-01-19,21,10,19,18,39,40.805159,-73.954692,1,1,Healthy,21:10,5,Saturday,winter,3


In [25]:
dock_sample2 = dock_sample.groupby(['dock_status'])[['dock_status']].count()

In [26]:
dock_sample2

,dock_status
dock_status,
Empty Alert,636124
Full Alert,234065
Healthy,424033


In [27]:
daily_temp = weather_df.groupby(["date", "hour"]).agg({
  "TMP" : ['min', 'max', 'mean'], 
  "wind_speed" : ["min", 'max', 'mean'], 
  "precipitation" : ["sum"]
  }).reset_index()
daily_temp.columns = ["_".join(x) for x in daily_temp.columns.ravel()]
daily_temp = daily_temp.rename(columns = {'date_': 'date', 'hour_':'hour'})

In [28]:
dock_sample['date'] = pd.to_datetime(dock_sample['date'])

In [29]:
dock_sample['month'] = dock_sample.date.dt.month

In [30]:
daily_temp.dtypes

date                  object
hour                   int64
TMP_min              float64
TMP_max              float64
TMP_mean             float64
wind_speed_min       float64
wind_speed_max       float64
wind_speed_mean      float64
precipitation_sum    float64
dtype: object

In [31]:
daily_temp['date'] = pd.to_datetime(daily_temp['date'])

In [32]:
combined = dock_sample.merge(daily_temp, on = ['date','hour'], how = 'inner')

In [33]:
pd.set_option('display.max_columns', None)

In [34]:
# Feature Engineering
combined['min'] = combined['minute'].apply(lambda x: 0 if x < 30 else 30)
combined['season2'] = combined['season'].apply(lambda x: 1 if x == 'spring' else 2 if x == 'summer' else \
                                             3 if x == 'fall' else 4)

In [22]:
# Random Forest Classifier

In [35]:
X = combined[['dock_id', 'hour', 'dayofweek', 'TMP_mean']]

In [36]:
y = combined['dock_status']

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 80% training and 20% test

In [38]:
clf=RandomForestClassifier(bootstrap=True, n_estimators=90, max_depth = 22, criterion = 'gini', class_weight = 'balanced')

In [39]:
clf.fit(X_train,y_train)

RandomForestClassifier(class_weight='balanced', max_depth=22, n_estimators=90)

In [40]:
y_pred=clf.predict(X_test)

In [45]:
y_pred

array(['Healthy', 'Empty Alert', 'Full Alert', ..., 'Full Alert',
       'Healthy', 'Full Alert'], dtype=object)

In [41]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5738115208017462


In [43]:
import pickle

In [44]:
Pkl_Filename = "clf.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(clf, file)

In [59]:
randomForest = ensemble.RandomForestClassifier()

# fit RF model on training set
randomForest.set_params(random_state=0)

randomForest.fit(X_train, y_train) 

print("The training error is: %.5f" % (1 - randomForest.score(X_train, y_train)))
print("The test     error is: %.5f" % (1 - randomForest.score(X_test, y_test)))

The training error is: 0.22089
The test     error is: 0.43844


In [60]:
# set the parameter grid
grid_para_forest = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 31),
    'n_estimators': range(10, 110, 10)
}

In [34]:
# run the RF classifier through the grid search
grid_search_forest = ms.GridSearchCV(randomForest, grid_para_forest, scoring='accuracy', cv=5, n_jobs=-1)
%time grid_search_forest.fit(X_train, y_train)

/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


CPU times: user 7min 16s, sys: 1min 7s, total: 8min 23s
Wall time: 8h 24min 54s


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 31),
                         'n_estimators': range(10, 110, 10)},
             scoring='accuracy')

In [35]:
# get the best parameters
grid_search_forest.best_params_

{'criterion': 'gini', 'max_depth': 22, 'n_estimators': 90}

In [36]:
# get the best score
grid_search_forest.best_score_

0.5614656451184326

In [37]:
# get the training/test errors
print("The training error is: %.5f" % (1 - grid_search_forest.best_estimator_.score(X_train, y_train)))
print("The test     error is: %.5f" % (1 - grid_search_forest.best_estimator_.score(X_test, y_test)))

The training error is: 0.13883
The test     error is: 0.43265


In [42]:
import joblib

In [61]:
filename = 'rf.save'
joblib.dump(clf, filename)

['rf.save']

In [ ]:
# (3)
feature_importance = list(zip(oj_data.columns, randomForest.feature_importances_))
dtype = [('feature', 'S10'), ('importance', 'float')]
feature_importance = np.array(feature_importance, dtype=dtype)
feature_sort = np.sort(feature_importance, order='importance')[::-1]
[i for (i, j) in feature_sort[0:5]]